In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Generate sample data for 1000 customers
np.random.seed(123)
customer_ids = range(1001, 2001)
loan_amounts = np.random.choice(range(10000, 50000, 1000), size=1000)
tenures = np.random.choice([12, 24, 36, 48, 60], size=1000)
acceptances = np.random.choice(["Accept", "Decline"], size=1000, p=[0.6, 0.4])

data = {'CustomerID': customer_ids, 'LoanAmount': loan_amounts, 'Tenure': tenures, 'Acceptance': acceptances}
df = pd.DataFrame(data)

df.head()

,CustomerID,LoanAmount,Tenure,Acceptance
0,1001,12000,60,Decline
1,1002,38000,24,Accept
2,1003,44000,48,Accept
3,1004,48000,60,Accept
4,1005,27000,36,Accept


In [9]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Tenure': [12, 24, 18, 36, 12],
    'InterestRate': [5.5, 6.2, 4.8, 7.0, 5.0],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = ['Offer A', 'Offer B', 'Offer C', 'Default Offer']

# Define the threshold for the number of offers
threshold_offers = 3
df.head()

,CustomerID,LoanAmount,Tenure,InterestRate,Acceptance
0,1,10000,12,5.5,Accept
1,2,15000,24,6.2,Decline
2,3,20000,18,4.8,Accept
3,4,12000,36,7.0,Decline
4,5,8000,12,5.0,Accept


In [10]:
import numpy as np

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state
    done = False
    
    while not done:
        # Choose an action based on epsilon-greedy policy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(num_actions)
        else:
            action = np.argmax(Q[state, :])
        
        # Take the action and observe the next state and reward
        next_state = state + 1
        reward = 1 if df['Acceptance'][state] == 'Accept' else -1
        
        # Update the Q-table using the Q-learning equation
        Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
        
        state = next_state
        
        # Check if the episode is done
        if state >= len(df) or state >= threshold_offers:
            done = True


In [11]:
# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]

# Add a column to the dataframe to store the recommended offer
df['RecommendedOffer'] = df.index.map(get_recommendation)


In [12]:
# Add a column to store the final output (offer details) for accepted offers
df['FinalOutput'] = ''

# Fill the final output column for accepted offers
df.loc[df['Acceptance'] == 'Accept', 'FinalOutput'] = (
    'Offer: ' + df['RecommendedOffer'] +
    ', Customer ID: ' + df['CustomerID'].astype(str)
)

# Print the dataframe with the final output
print(df[['CustomerID', 'RecommendedOffer', 'FinalOutput']])


   CustomerID RecommendedOffer                     FinalOutput
0           1          Offer A  Offer: Offer A, Customer ID: 1
1           2          Offer B                                
2           3          Offer A  Offer: Offer A, Customer ID: 3
3           4          Offer A                                
4           5          Offer A  Offer: Offer A, Customer ID: 5


In [14]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {'Tenure': 12, 'InterestRate': 5.5},
    'Offer B': {'Tenure': 24, 'InterestRate': 6.2},
    'Offer C': {'Tenure': 18, 'InterestRate': 4.8},
    'Default Offer': {'Tenure': 36, 'InterestRate': 7.0}
}

# Define the threshold for the number of offers
threshold_offers = 3

df.head()

,CustomerID,LoanAmount,Acceptance
0,1,10000,Accept
1,2,15000,Decline
2,3,20000,Accept
3,4,12000,Decline
4,5,8000,Accept


In [15]:
import numpy as np

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state
    done = False
    
    while not done:
        # Choose an action based on epsilon-greedy policy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(num_actions)
        else:
            action = np.argmax(Q[state, :])
        
        # Take the action and observe the next state and reward
        next_state = state + 1
        reward = 1 if df['Acceptance'][state] == 'Accept' else -1
        
        # Update the Q-table using the Q-learning equation
        Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
        
        state = next_state
        
        # Check if the episode is done
        if state >= len(df) or state >= threshold_offers:
            done = True


In [16]:
# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return list(offers.keys())[action]

# Add columns to the dataframe to store the recommended offer details
df['RecommendedOffer'] = df.index.map(get_recommendation)
df['RecommendedTenure'] = df['RecommendedOffer'].map(lambda x: offers[x]['Tenure'])
df['RecommendedInterestRate'] = df['RecommendedOffer'].map(lambda x: offers[x]['InterestRate'])


In [17]:
# Add columns to store the final output (offer details) for accepted offers
df['FinalOutput'] = ''
df['FinalOutput'] = df.apply(lambda row: 'Offer: ' + row['RecommendedOffer'] +
                            ', Tenure: ' + str(row['RecommendedTenure']) +
                            ', Interest Rate: ' + str(row['RecommendedInterestRate']) +
                            ', Customer ID: ' + str(row['CustomerID'])
                            if row['Acceptance'] == 'Accept' else '', axis=1)

# Print the dataframe with the final output
print(df[['CustomerID', 'RecommendedOffer', 'RecommendedTenure', 'RecommendedInterestRate', 'FinalOutput']])


   CustomerID RecommendedOffer  RecommendedTenure  RecommendedInterestRate  \
0           1          Offer A                 12                      5.5   
1           2          Offer A                 12                      5.5   
2           3          Offer A                 12                      5.5   
3           4          Offer A                 12                      5.5   
4           5          Offer A                 12                      5.5   

                                         FinalOutput  
0  Offer: Offer A, Tenure: 12, Interest Rate: 5.5...  
1                                                     
2  Offer: Offer A, Tenure: 12, Interest Rate: 5.5...  
3                                                     
4  Offer: Offer A, Tenure: 12, Interest Rate: 5.5...  


In [18]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {'Tenure': 12, 'InterestRate': 5.5},
    'Offer B': {'Tenure': 24, 'InterestRate': 6.2},
    'Offer C': {'Tenure': 18, 'InterestRate': 4.8},
    'Default Offer': {'Tenure': 36, 'InterestRate': 7.0}
}

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Q-learning algorithm
for episode in range(num_episodes):
    state = 0  # Initial state
    done = False
    num_offers = 0
    
    while not done:
        # Choose an action based on epsilon-greedy policy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(num_actions)
        else:
            action = np.argmax(Q[state, :])
        
        # Take the action and observe the next state and reward
        next_state = state + 1
        reward = 1 if df['Acceptance'][state] == 'Accept' else -1
        
        # Update the Q-table using the Q-learning equation
        Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
        
        state = next_state
        num_offers += 1
        
        # Check if the episode is done
        if num_offers >= threshold_offers or state >= len(df):
            done = True
        elif df['Acceptance'][state] == 'Accept':
            done = True
        elif num_offers < threshold_offers and state < len(df):
            continue
        
        # Offer the default offer if the threshold is reached
        if num_offers >= threshold_offers:
            state += 1
        
# Function to get the recommended offer for a given state
def get_recommendation(state):
    if state < len(df):
        action = np.argmax(Q[state, :])
        return list(offers.keys())[action]
    else:
        return 'Default Offer'

# Add columns to the dataframe to store the recommended offer details
df['RecommendedOffer'] = df.index.map(get_recommendation)
df['RecommendedTenure'] = df['RecommendedOffer'].map(lambda x: offers[x]['Tenure'])
df['RecommendedInterestRate'] = df['RecommendedOffer'].map(lambda x: offers[x]['InterestRate'])

# Add columns to store the final output (offer details) for accepted offers
df['FinalOutput'] = ''
df.loc[df['Acceptance'] == 'Accept', 'FinalOutput'] = (
    'Offer: ' + df['RecommendedOffer'] +
    ', Tenure: ' + df['RecommendedTenure'].astype(str) +
    ', Interest Rate: ' + df['RecommendedInterestRate'].astype(str) +
    ', Customer ID: ' + df['CustomerID'].astype(str)
)

# Print the dataframe with the final output
print(df[['CustomerID', 'RecommendedOffer', 'RecommendedTenure', 'RecommendedInterestRate', 'FinalOutput']])


   CustomerID RecommendedOffer  RecommendedTenure  RecommendedInterestRate  \
0           1    Default Offer                 36                      7.0   
1           2          Offer A                 12                      5.5   
2           3          Offer A                 12                      5.5   
3           4          Offer A                 12                      5.5   
4           5          Offer A                 12                      5.5   

                                         FinalOutput  
0  Offer: Default Offer, Tenure: 36, Interest Rat...  
1                                                     
2  Offer: Offer A, Tenure: 12, Interest Rate: 5.5...  
3                                                     
4  Offer: Offer A, Tenure: 12, Interest Rate: 5.5...  


In [20]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {'Tenure': 12, 'InterestRate': 5.5},
    'Offer B': {'Tenure': 24, 'InterestRate': 6.2},
    'Offer C': {'Tenure': 18, 'InterestRate': 4.8},
    'Default Offer': {'Tenure': 36, 'InterestRate': 7.0}
}

# Define the threshold for the number of offers
threshold_offers = 3

# Function to offer recommendations to the customer
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    
    recommended_offer = None
    for _ in range(threshold_offers):
        if recommended_offer is None:
            recommended_offer = get_recommendation(customer_data)
        else:
            recommended_offer = get_next_offer(recommended_offer)
        
        print(f"Offer: {recommended_offer}")
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
    
    if recommended_offer is None:
        recommended_offer = get_recommendation(customer_data)
        print(f"Default Offer: {recommended_offer}")
        print("Default offer presented.")
    
    return recommended_offer


# Function to get the recommended offer for a given customer
def get_recommendation(customer_data):
    # Implement your recommendation logic here
    # You can use customer_data and offers dictionary to make the recommendation
    # This is a sample implementation that recommends the first offer
    return list(offers.keys())[0]


# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    offer_keys = list(offers.keys())
    current_offer_index = offer_keys.index(current_offer)
    
    # Return the next offer in the list
    if current_offer_index < len(offer_keys) - 1:
        return offer_keys[current_offer_index + 1]
    
    # If the current offer is the last offer, return the default offer
    return 'Default Offer'


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Enter customer response (Accept/Decline): Accept
Offer accepted!
Recommended Offer for Customer 3: Offer A


In [21]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    'Offer B': {
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    'Offer C': {
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    'Default Offer': {
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
}

# Define the threshold for the number of offers
threshold_offers = 3

# Function to offer recommendations to the customer
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    
    recommended_offer = None
    for _ in range(threshold_offers):
        if recommended_offer is None:
            recommended_offer = get_recommendation(customer_data)
        else:
            recommended_offer = get_next_offer(recommended_offer)
        
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
    
    if recommended_offer is None:
        recommended_offer = get_recommendation(customer_data)
        print(f"Default Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to get the recommended offer for a given customer
def get_recommendation(customer_data):
    # Implement your recommendation logic here
    # You can use customer_data and offers dictionary to make the recommendation
    # This is a sample implementation that recommends the first offer
    return list(offers.keys())[0]


# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    offer_keys = list(offers.keys())
    current_offer_index = offer_keys.index(current_offer)
    
    # Return the next offer in the list
    if current_offer_index < len(offer_keys) - 1:
        return offer_keys[current_offer_index + 1]
    
    # If the current offer is the last offer, return the default offer
    return 'Default Offer'


# Function to print offer details
def print_offer_details(offer):
    offer_details = offers[offer]
    print(f"Offer: {offer}")
    print(f" - Tenure: {offer_details['Tenure']}")
    print(f" - Interest Rate: {offer_details['InterestRate']}")
    print(f" - Processing Fee: {offer_details['ProcessingFee']}")
    print(f" - Insurance Fee: {offer_details['InsuranceFee']}")


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer B
Offer: Offer B
 - Tenure: 24
 - Interest Rate: 6.2
 - Processing Fee: 150
 - Insurance Fee: 75
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer C
Offer: Offer C
 - Tenure: 18
 - Interest Rate: 4.8
 - Processing Fee: 120
 - Insurance Fee: 60
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Recommended Offer for Customer 3: Offer C


In [25]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    'Offer B': {
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    'Offer C': {
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    'Default Offer': {
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
}

# Define the threshold for the number of offers
threshold_offers = 3

# Function to offer recommendations to the customer
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    
    recommended_offer = None
    num_offers = 0
    
    while num_offers < threshold_offers:
        if recommended_offer is None:
            recommended_offer = get_recommendation(customer_data)
        else:
            recommended_offer = get_next_offer(recommended_offer)
        
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = get_recommendation(customer_data)
        print(f"Default Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer



# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    if current_offer == 'Default Offer':
        return 'Default Offer'
    
    offer_keys = list(offers.keys())
    current_offer_index = offer_keys.index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offer_keys) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offer_keys[current_offer_index + 1]


# Function to offer recommendations to the customer
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    
    recommended_offer = None
    num_offers = 0
    
    while num_offers < threshold_offers:
        if recommended_offer is None:
            recommended_offer = get_recommendation(customer_data)
        else:
            recommended_offer = get_next_offer(recommended_offer)
        
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer




# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        print(f"Default Offer")
    else:
        offer_details = offers[offer]
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")



# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer B
Offer: Offer B
 - Tenure: 24
 - Interest Rate: 6.2
 - Processing Fee: 150
 - Insurance Fee: 75
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer C
Offer: Offer C
 - Tenure: 18
 - Interest Rate: 4.8
 - Processing Fee: 120
 - Insurance Fee: 60
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [26]:
import pandas as pd

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = {
    'Offer A': {
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    'Offer B': {
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    'Offer C': {
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    'Default Offer': {
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
}

# Define the threshold for the number of offers
threshold_offers = 3

# Function to offer recommendations to the customer
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    
    recommended_offer = None
    num_offers = 0
    
    while num_offers < threshold_offers:
        if recommended_offer is None:
            recommended_offer = get_recommendation(customer_data)
        else:
            recommended_offer = get_next_offer(recommended_offer)
        
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to get the recommended offer for a given customer
def get_recommendation(customer_data):
    # Implement your recommendation logic here
    # You can use customer_data and offers dictionary to make the recommendation
    # This is a sample implementation that recommends the first offer
    return list(offers.keys())[0]


# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    if current_offer == 'Default Offer':
        return 'Default Offer'
    
    offer_keys = list(offers.keys())
    current_offer_index = offer_keys.index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offer_keys) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offer_keys[current_offer_index + 1]


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = offers[offer]
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = offers[offer]
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer B
Offer: Offer B
 - Tenure: 24
 - Interest Rate: 6.2
 - Processing Fee: 150
 - Insurance Fee: 75
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer C
Offer: Offer C
 - Tenure: 18
 - Interest Rate: 4.8
 - Processing Fee: 120
 - Insurance Fee: 60
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [29]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offers[current_offer_index + 1]['Name']

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    
    while num_offers < threshold_offers:
        recommended_offer = get_recommendation(state)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            Q[state, [offer['Name'] for offer in offers].index(recommended_offer)] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, [offer['Name'] for offer in offers].index(recommended_offer)])
            state = next_state
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [30]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offers[current_offer_index + 1]['Name']

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    
    while num_offers < threshold_offers:
        recommended_offer = get_recommendation(state)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            Q[state, [offer['Name'] for offer in offers].index(recommended_offer)] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, [offer['Name'] for offer in offers].index(recommended_offer)])
            state = next_state
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [31]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offers[current_offer_index + 1]['Name']

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    
    while num_offers < threshold_offers:
        recommended_offer = get_recommendation(state)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            for action in range(num_actions):
                Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            state = next_state
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [32]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer and customer's response
def get_next_offer(current_offer, response):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # If the response is "Decline", return the next offer
    if response.lower() == 'decline':
        return offers[current_offer_index + 1]['Name']
    
    # If the response is "Accept", return the current offer
    return current_offer

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    
    while num_offers < threshold_offers:
        recommended_offer = get_recommendation(state)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_offer = get_next_offer(recommended_offer, response)
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            Q[state, [offer['Name'] for offer in offers].index(recommended_offer)] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, [offer['Name'] for offer in offers].index(recommended_offer)])
            state = next_state
            recommended_offer = next_offer
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Accept
Offer accepted!
Recommended Offer for Customer 3: Offer A


In [33]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer
def get_next_offer(current_offer):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # Return the next offer in the list
    return offers[current_offer_index + 1]['Name']

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    
    while num_offers < threshold_offers:
        recommended_offer = get_recommendation(state)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_offer = get_next_offer(recommended_offer)
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            for action in range(num_actions):
                Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            state = next_state
            recommended_offer = next_offer
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Accpet
Offer declined. Offering the next set of offers.
Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Accept
Offer accepted!
Recommended Offer for Customer 3: Offer A


In [35]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer and customer's response
def get_next_offer(current_offer, response):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # If the response is "Decline", return the next offer
    if response.lower() == 'decline':
        return offers[current_offer_index + 1]['Name']
    
    # If the response is "Accept", return the current offer
    return current_offer

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    current_offer = None
    
    while num_offers < threshold_offers:
        if current_offer is None:
            recommended_offer = get_recommendation(state)
        else:
            recommended_offer = get_next_offer(current_offer, response)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_offer = get_next_offer(recommended_offer, response)
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            Q[state, [offer['Name'] for offer in offers].index(recommended_offer)] += learning_rate * (
                    reward + discount_factor * np.max(Q[next_state, :]) - Q[state, [offer['Name'] for offer in offers].index(recommended_offer)])
            state = next_state
            current_offer = next_offer
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer C
Offer: Offer C
 - Tenure: 18
 - Interest Rate: 4.8
 - Processing Fee: 120
 - Insurance Fee: 60

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [36]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer and customer's response
def get_next_offer(current_offer, response):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # If the response is "Decline", return the next offer
    if response.lower() == 'decline':
        return offers[current_offer_index + 1]['Name']
    
    # If the response is "Accept", return the current offer
    return current_offer

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    current_offer = None
    
    while num_offers < threshold_offers:
        if current_offer is None:
            recommended_offer = get_recommendation(state)
        else:
            recommended_offer = get_next_offer(current_offer, response)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_offer = get_next_offer(recommended_offer, response)
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            for action in range(num_actions):
                Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            state = next_state
            current_offer = recommended_offer
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer B
Offer: Offer B
 - Tenure: 24
 - Interest Rate: 6.2
 - Processing Fee: 150
 - Insurance Fee: 75

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Offer: Offer C
Offer: Offer C
 - Tenure: 18
 - Interest Rate: 4.8
 - Processing Fee: 120
 - Insurance Fee: 60

Enter customer response (Accept/Decline): Decline
Offer declined. Offering the next set of offers.
Default Offer
Default Offer Details:
 - Tenure: 36
 - Interest Rate: 7.0
 - Processing Fee: 200
 - Insurance Fee: 100
Default offer presented.
Recommended Offer for Customer 3: Default Offer


In [37]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id, response):
    # Perform the recommendation logic here based on the customer and response

    # Replace the following lines with your recommendation logic
    if response.lower() == 'accept':
        recommended_offer = "Offer A"
    else:
        recommended_offer = "Offer B"

    return recommended_offer

# Route for the home page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        customer_id = request.form.get('customer_id')
        response = request.form.get('response')
        recommended_offer = offer_recommendation(customer_id, response)
        return render_template('result1.html', offer=recommended_offer)
    else:
        return render_template('index1.html')

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [40]:
import pandas as pd
import numpy as np
import random

# Define the sample dataset with 'InterestData'
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept'],
    'InterestData': ['Low', 'Medium', 'High', 'Low', 'Medium']
}

df = pd.DataFrame(data)

# Add the 'InterestRate' column to the DataFrame with interest rates ranging from 0.1 to 1.5
np.random.seed(42)  # For reproducibility
df['InterestRate'] = np.random.uniform(0.1, 1.5, size=len(df))

# Define the action space (original offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 0.55,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 0.62,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 0.48,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 0.7,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Analyze historical data and create bins based on acceptance rates
acceptance_rates = df.groupby('Acceptance').size() / len(df)
num_bins = 3
bin_edges = [0, 0.3333, 0.6666, 1]
bin_labels = [f"Bin {i+1}" for i in range(num_bins)]

df['AcceptanceBin'] = pd.cut(df['Acceptance'].map(acceptance_rates), bins=bin_edges, labels=bin_labels, include_lowest=True, duplicates='drop')

# Function to create new offers based on bins and historical acceptance rates
def create_new_offer(bin_label):
    bin_data = df[df['AcceptanceBin'] == bin_label]
    selected_offer = random.choice(bin_data.index)
    new_offer = offers[selected_offer].copy()
    # Modify tenor, loan amount, and interest rate based on historical acceptance rate
    new_offer['Tenure'] = int(new_offer['Tenure'] * acceptance_rates[bin_label])
    new_offer['LoanAmount'] = int(new_offer['LoanAmount'] * acceptance_rates[bin_label])
    new_offer['InterestRate'] = round(new_offer['InterestRate'] * acceptance_rates[bin_label], 2)
    return new_offer

# Create new offers for each bin
new_offers = [create_new_offer(bin_label) for bin_label in bin_labels]

# Print the new offers
print("New Offers:")
for i, new_offer in enumerate(new_offers):
    print(f"Bin {i+1}:")
    print(f" - Offer Name: {new_offer['Name']}")
    print(f" - Tenure: {new_offer['Tenure']} months")
    print(f" - Loan Amount: {new_offer['LoanAmount']}")
    print(f" - Interest Rate: {new_offer['InterestRate']}")
    print()

# ... (Rest of the code remains the same)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [50]:
import pandas as pd
import numpy as np
import random

# Define the sample dataset with 'InterestData'
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept'],
    'InterestData': ['Low', 'Medium', 'High', 'Low', 'Medium']
}

df = pd.DataFrame(data)

# Add the 'InterestRate' column to the DataFrame with interest rates ranging from 0.1 to 1.5
np.random.seed(42)  # For reproducibility
df['InterestRate'] = np.random.uniform(0.1, 0.6, size=len(df))

# Define the action space (original offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 0.55,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 0.62,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 0.48,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 0.7,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Analyze historical data and create bins based on acceptance rates
acceptance_rates = df.groupby('Acceptance').size() / len(df)
num_bins = 9
bin_edges = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
bin_labels = [f"Bin {i+1}" for i in range(num_bins)]

df['AcceptanceBin'] = pd.cut(df['Acceptance'].map(acceptance_rates), bins=bin_edges, labels=bin_labels, include_lowest=True, duplicates='drop')

df.head()

,CustomerID,LoanAmount,Acceptance,InterestData,InterestRate,AcceptanceBin
0,1,10000,Accept,Low,0.287270,NaN
1,2,15000,Decline,Medium,0.575357,Bin 6
2,3,20000,Accept,High,0.465997,NaN
3,4,12000,Decline,Low,0.399329,Bin 6
4,5,8000,Accept,Medium,0.178009,NaN


In [ ]:
# Function to create new offers based on bins and historical acceptance rates
def create_new_offer(bin_label):
    bin_data = df[df['AcceptanceBin'] == bin_label]
    if bin_data.empty:
        return create_default_offer(bin_label)
    selected_offer = random.choice(bin_data.index)
    new_offer = offers[selected_offer].copy()
    # Modify tenor, loan amount, and interest rate based on historical acceptance rate
    new_offer['Tenure'] = int(new_offer['Tenure'] * acceptance_rates[bin_label])
    new_offer['LoanAmount'] = int(new_offer['LoanAmount'] * acceptance_rates[bin_label])
    new_offer['InterestRate'] = round(new_offer['InterestRate'] * acceptance_rates[bin_label], 2)
    return new_offer

# Function to create a default offer when a bin is empty
def create_default_offer(bin_label):
    default_offer = offers[-1].copy()  # Use the last offer as the default
    default_offer['Tenure'] = int(default_offer['Tenure'] * acceptance_rates[bin_label])
    default_offer['LoanAmount'] = int(default_offer['LoanAmount'] * acceptance_rates[bin_label])
    default_offer['InterestRate'] = round(default_offer['InterestRate'] * acceptance_rates[bin_label], 2)
    return default_offer

# Create new offers for each bin
new_offers = [create_new_offer(bin_label) for bin_label in bin_labels]

# Print the new offers
print("New Offers:")
for i, new_offer in enumerate(new_offers):
    print(f"Bin {i+1}:")
    print(f" - Offer Name: {new_offer['Name']}")
    print(f" - Tenure: {new_offer['Tenure']} months")
    print(f" - Loan Amount: {new_offer['LoanAmount']}")
    print(f" - Interest Rate: {new_offer['InterestRate']}")
    print()


In [51]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'InterestRate': [0.8, 1.2, 0.9, 1.4, 1.1],  # Sample interest rates for each customer
    'Tenor': [12, 24, 18, 12, 36],  # Sample tenure for each customer
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Function to get the customer's decision based on the acceptance status
def get_decision(acceptance_status):
    return 'Accepted' if acceptance_status == 'Accept' else 'Declined'

# Add the Decision column to the DataFrame based on the Acceptance column
df['Decision'] = df['Acceptance'].apply(get_decision)

# Display the new dataset
print(df)


   CustomerID  LoanAmount  InterestRate  Tenor Acceptance  Decision
0           1       10000           0.8     12     Accept  Accepted
1           2       15000           1.2     24    Decline  Declined
2           3       20000           0.9     18     Accept  Accepted
3           4       12000           1.4     12    Decline  Declined
4           5        8000           1.1     36     Accept  Accepted


In [52]:
import pandas as pd
import itertools

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'InterestRate': [0.8, 1.2, 0.9, 1.4, 1.1],  # Sample interest rates for each customer
    'Tenor': [12, 24, 18, 12, 36],  # Sample tenure for each customer
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Filter the DataFrame to include only rows where Decision is Accepted
accepted_offers_df = df[df['Acceptance'] == 'Accept'].copy()

# Create all possible combinations of InterestRate, Tenor, and LoanAmount
interest_rates = accepted_offers_df['InterestRate'].unique()
tenors = accepted_offers_df['Tenor'].unique()
loan_amounts = accepted_offers_df['LoanAmount'].unique()

combinations = list(itertools.product(interest_rates, tenors, loan_amounts))

# Create the offers DataFrame with all possible combinations
offers_data = {
    'InterestRate': [comb[0] for comb in combinations],
    'Tenor': [comb[1] for comb in combinations],
    'LoanAmount': [comb[2] for comb in combinations],
    'Decision': ['Accepted'] * len(combinations)
}

offers_df = pd.DataFrame(offers_data)

# Display the offers DataFrame
print(offers_df)


    InterestRate  Tenor  LoanAmount  Decision
0            0.8     12       10000  Accepted
1            0.8     12       20000  Accepted
2            0.8     12        8000  Accepted
3            0.8     18       10000  Accepted
4            0.8     18       20000  Accepted
5            0.8     18        8000  Accepted
6            0.8     36       10000  Accepted
7            0.8     36       20000  Accepted
8            0.8     36        8000  Accepted
9            0.9     12       10000  Accepted
10           0.9     12       20000  Accepted
11           0.9     12        8000  Accepted
12           0.9     18       10000  Accepted
13           0.9     18       20000  Accepted
14           0.9     18        8000  Accepted
15           0.9     36       10000  Accepted
16           0.9     36       20000  Accepted
17           0.9     36        8000  Accepted
18           1.1     12       10000  Accepted
19           1.1     12       20000  Accepted
20           1.1     12        800

In [54]:
import pandas as pd
import itertools

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'InterestRate': [0.8, 1.2, 0.9, 1.4, 1.1],  # Sample interest rates for each customer
    'Tenor': [12, 24, 18, 12, 36],  # Sample tenure for each customer
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Filter the DataFrame to include only rows where Decision is Accepted
accepted_offers_df = df[df['Acceptance'] == 'Accept'].copy()

# Create all possible combinations of InterestRate, Tenor, and LoanAmount
interest_rates = accepted_offers_df['InterestRate'].unique()
tenors = accepted_offers_df['Tenor'].unique()
loan_amounts = accepted_offers_df['LoanAmount'].unique()

combinations = list(itertools.product(interest_rates, tenors, loan_amounts))

# Create the offers DataFrame with all possible combinations
offers_data = {
    'InterestRate': [comb[0] for comb in combinations],
    'Tenor': [comb[1] for comb in combinations],
    'LoanAmount': [comb[2] for comb in combinations],
    'Decision': ['Accepted'] * len(combinations)
}

offers_df = pd.DataFrame(offers_data)

# Display the offers DataFrame
print(offers_df)


    InterestRate  Tenor  LoanAmount  Decision
0            0.8     12       10000  Accepted
1            0.8     12       20000  Accepted
2            0.8     12        8000  Accepted
3            0.8     18       10000  Accepted
4            0.8     18       20000  Accepted
5            0.8     18        8000  Accepted
6            0.8     36       10000  Accepted
7            0.8     36       20000  Accepted
8            0.8     36        8000  Accepted
9            0.9     12       10000  Accepted
10           0.9     12       20000  Accepted
11           0.9     12        8000  Accepted
12           0.9     18       10000  Accepted
13           0.9     18       20000  Accepted
14           0.9     18        8000  Accepted
15           0.9     36       10000  Accepted
16           0.9     36       20000  Accepted
17           0.9     36        8000  Accepted
18           1.1     12       10000  Accepted
19           1.1     12       20000  Accepted
20           1.1     12        800

In [55]:
# Sort the offers DataFrame by LoanAmount (descending), InterestRate (descending), and Tenor (descending)
sorted_offers_df = offers_df.sort_values(by=['LoanAmount', 'InterestRate', 'Tenor'], ascending=[False, False, False])

# Select the top 10 offers
top_10_offers = sorted_offers_df.head(10)

# Display the top 10 offers
print("Top 10 Offers:")
print(top_10_offers)


Top 10 Offers:
    InterestRate  Tenor  LoanAmount  Decision
25           1.1     36       20000  Accepted
22           1.1     18       20000  Accepted
19           1.1     12       20000  Accepted
16           0.9     36       20000  Accepted
13           0.9     18       20000  Accepted
10           0.9     12       20000  Accepted
7            0.8     36       20000  Accepted
4            0.8     18       20000  Accepted
1            0.8     12       20000  Accepted
24           1.1     36       10000  Accepted


In [56]:
# Create a list to store the offers as dictionaries
offers_list = []

# Iterate over the rows in the top_10_offers DataFrame and convert each row into a dictionary
for idx, row in top_10_offers.iterrows():
    offer_dict = {
        'Name': f'Offer {idx+1}',  # Create a unique offer name based on the row index
        'Tenure': row['Tenor'],
        'LoanAmount': row['LoanAmount'],
        'InterestRate': row['InterestRate']
    }
    offers_list.append(offer_dict)

# Display the list of offers as dictionaries
print(offers_list)


[{'Name': 'Offer 26', 'Tenure': 36, 'LoanAmount': 20000, 'InterestRate': 1.1}, {'Name': 'Offer 23', 'Tenure': 18, 'LoanAmount': 20000, 'InterestRate': 1.1}, {'Name': 'Offer 20', 'Tenure': 12, 'LoanAmount': 20000, 'InterestRate': 1.1}, {'Name': 'Offer 17', 'Tenure': 36, 'LoanAmount': 20000, 'InterestRate': 0.9}, {'Name': 'Offer 14', 'Tenure': 18, 'LoanAmount': 20000, 'InterestRate': 0.9}, {'Name': 'Offer 11', 'Tenure': 12, 'LoanAmount': 20000, 'InterestRate': 0.9}, {'Name': 'Offer 8', 'Tenure': 36, 'LoanAmount': 20000, 'InterestRate': 0.8}, {'Name': 'Offer 5', 'Tenure': 18, 'LoanAmount': 20000, 'InterestRate': 0.8}, {'Name': 'Offer 2', 'Tenure': 12, 'LoanAmount': 20000, 'InterestRate': 0.8}, {'Name': 'Offer 25', 'Tenure': 36, 'LoanAmount': 10000, 'InterestRate': 1.1}]
